# Ham or Spam?

In [1]:
# when installing nltk for the first time we need to also download a few built in libraries
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import string

Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vincent/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/vincent/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vincent/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Avant de pouvoir ouvrir le fichier CSV, il va falloir le nettoyer. En effet, le fichier ne respecte pas un format consistant.

In [2]:
def fix_broken_csv():
    """Fonction qui répare le csv et stocke les données 'propres' dans un fichier clean_emails.csv"""
    
    data = ["text,spam\n"]

    with open("emails.csv", "r") as file:

        lines = file.readlines()

        for line in lines:
            label = line.split(",")[-1]
            text = line.removesuffix(label)

            clean_label = None

            for letter in label:
                if letter.isdigit(): 
                    clean_label = letter
                    break
            if not clean_label : continue

            clean_text = text.rstrip(",").strip('"')
            clean_text = '"' + clean_text + '"'
            data.append(clean_text + ","  + clean_label + "\n")
            
        print(data[:10])

        with open("clean_emails.csv", "w") as clean_file:
            clean_file.writelines(data)
        
            
fix_broken_csv()      


['text,spam\n', '"Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . afforda

In [3]:
import pandas as pd

df = pd.read_csv("clean_emails.csv")

df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


The dataset is made up of email that are classified as ham [0] or spam[1]. You need to clean the dataset before training a prediction model.

## Remove Punctuation

👇 Create a function to remove the punctuation. Apply it to the entire data and add the output as a new column in the dataframe called `clean_text`

In [4]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

df["clean_text"] = df['text'].apply(remove_punctuations)
print(df.head())

                                                text  spam  \
0  Subject: naturally irresistible your corporate...     1   
1  Subject: the stock trading gunslinger  fanny i...     1   
2  Subject: unbelievable new homes made easy  im ...     1   
3  Subject: 4 color printing special  request add...     1   
4  Subject: do not have money , get software cds ...     1   

                                          clean_text  
0  Subject naturally irresistible your corporate ...  
1  Subject the stock trading gunslinger  fanny is...  
2  Subject unbelievable new homes made easy  im w...  
3  Subject 4 color printing special  request addi...  
4  Subject do not have money  get software cds fr...  


## Lower Case

👇 Create a function to lower case the text. Apply it to `clean_text`

In [5]:
def lower_all(text): return text.lower()
df["clean_text"] = df["clean_text"].apply(lower_all)
print(df.head())

                                                text  spam  \
0  Subject: naturally irresistible your corporate...     1   
1  Subject: the stock trading gunslinger  fanny i...     1   
2  Subject: unbelievable new homes made easy  im ...     1   
3  Subject: 4 color printing special  request add...     1   
4  Subject: do not have money , get software cds ...     1   

                                          clean_text  
0  subject naturally irresistible your corporate ...  
1  subject the stock trading gunslinger  fanny is...  
2  subject unbelievable new homes made easy  im w...  
3  subject 4 color printing special  request addi...  
4  subject do not have money  get software cds fr...  


## Remove Numbers

👇 Create a function to remove numbers from the text. Apply it to `clean_text`

In [6]:
def remove_numbers(text):
    
    processed_text = ""
    for letter in text:
        if not letter.isdigit():
            processed_text += letter
    return processed_text

df["clean_text"] = df["clean_text"].apply(remove_numbers)
print(df.head())

                                                text  spam  \
0  Subject: naturally irresistible your corporate...     1   
1  Subject: the stock trading gunslinger  fanny i...     1   
2  Subject: unbelievable new homes made easy  im ...     1   
3  Subject: 4 color printing special  request add...     1   
4  Subject: do not have money , get software cds ...     1   

                                          clean_text  
0  subject naturally irresistible your corporate ...  
1  subject the stock trading gunslinger  fanny is...  
2  subject unbelievable new homes made easy  im w...  
3  subject  color printing special  request addit...  
4  subject do not have money  get software cds fr...  


## Remove StopWords

👇 Create a function to remove stopwords from the text. Apply it to `clean_text`.

In [7]:
stop = stopwords.words("english")

def remove_stopwords(text):
    
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stop]
    return " ".join(tokens_without_sw)

df["clean_text"] = df["clean_text"].apply(remove_stopwords)
print(df.head())

                                                text  spam  \
0  Subject: naturally irresistible your corporate...     1   
1  Subject: the stock trading gunslinger  fanny i...     1   
2  Subject: unbelievable new homes made easy  im ...     1   
3  Subject: 4 color printing special  request add...     1   
4  Subject: do not have money , get software cds ...     1   

                                          clean_text  
0  subject naturally irresistible corporate ident...  
1  subject stock trading gunslinger fanny merrill...  
2  subject unbelievable new homes made easy im wa...  
3  subject color printing special request additio...  
4  subject money get software cds software compat...  


## Lemmatize

👇 Create a function to lemmatize the text. Make sure the output is a single string, not a list of words. Apply it to `clean_text`.

In [8]:
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    
    words = text.split(" ")
    lem_words = ""
    for w in words:
        lem_words += lemmatizer.lemmatize(w)
        lem_words += " "
    return lem_words.strip(" ")

df["clean_text"] = df["clean_text"].apply(lemmatize)
print(df.head())

                                                text  spam  \
0  Subject: naturally irresistible your corporate...     1   
1  Subject: the stock trading gunslinger  fanny i...     1   
2  Subject: unbelievable new homes made easy  im ...     1   
3  Subject: 4 color printing special  request add...     1   
4  Subject: do not have money , get software cds ...     1   

                                          clean_text  
0  subject naturally irresistible corporate ident...  
1  subject stock trading gunslinger fanny merrill...  
2  subject unbelievable new home made easy im wan...  
3  subject color printing special request additio...  
4  subject money get software cd software compati...  


## Bag-of-words Modelling

👇 Vectorize the `clean_text` to a Bag-of-Words representation with a default CountVectorizer . Save as `X_bow`.

In [9]:
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(df["clean_text"])
print(X_bow)

  (0, 26346)	1
  (0, 18141)	1
  (0, 13914)	1
  (0, 5876)	1
  (0, 12861)	1
  (0, 16075)	1
  (0, 22382)	1
  (0, 11915)	1
  (0, 22498)	1
  (0, 5185)	3
  (0, 16556)	2
  (0, 10760)	1
  (0, 26584)	1
  (0, 13366)	1
  (0, 13957)	1
  (0, 11375)	2
  (0, 4059)	1
  (0, 15900)	4
  (0, 26327)	1
  (0, 26010)	1
  (0, 19515)	1
  (0, 29751)	2
  (0, 16351)	2
  (0, 26992)	1
  (0, 17828)	2
  :	:
  (5727, 6145)	1
  (5727, 8333)	1
  (5727, 8013)	1
  (5727, 11488)	1
  (5727, 17618)	1
  (5727, 3942)	1
  (5727, 16528)	1
  (5727, 1926)	9
  (5727, 25055)	1
  (5727, 8867)	1
  (5727, 8676)	2
  (5727, 12574)	1
  (5727, 9597)	1
  (5727, 15280)	1
  (5727, 25886)	1
  (5727, 15455)	1
  (5727, 29074)	1
  (5727, 4586)	1
  (5727, 24294)	1
  (5727, 9037)	5
  (5727, 29896)	1
  (5727, 24475)	1
  (5727, 12759)	1
  (5727, 27797)	1
  (5727, 7075)	1


👇 Cross-validate a MultinomialNB model with the Bag-of-words. Score the model's accuracy.

In [10]:
print("Accuracies with cross_validation:")
model = MultinomialNB()
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
clf = model
print(cross_val_score(clf, X_bow, df["spam"], cv=k_fold, n_jobs=1))


Accuracy with cross_validation:
[0.9930192  0.98254799 0.9895288  0.991274   0.991274   0.9947644
 0.991274   0.9895288  0.98776224 0.99125874]


⚠️ Please push the exercise once you are done 🙃

## 🏁 